# 多种格式数据加载、处理与存储
实际的场景中，我们会在不同的地方遇到各种不同的数据格式（比如大家熟悉的csv与txt，比如网页HTML格式，比如XML格式），我们来一起看看python如何和这些格式的数据打交道。

寒小阳
2016-08

In [1]:
from __future__ import division
from numpy.random import randn
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas as pd
np.set_printoptions(precision=4)

In [ ]:
%pwd

## 1.各式各样的文本数据

### 1.1 CSV与TXT读取

In [2]:
!cat data1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [3]:
df = pd.read_csv('data1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
pd.read_table('data1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [5]:
!cat data2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [7]:
pd.read_csv('data2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
pd.read_csv('data2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [9]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('data2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [10]:
!cat csv_mindex.csv
parsed = pd.read_csv('csv_mindex.csv', index_col=['key1', 'key2'])
parsed

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [12]:
list(open('data3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [13]:
result = pd.read_table('data3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [14]:
!cat data4.csv
pd.read_csv('data4.csv', skiprows=[0, 2, 3])

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [15]:
!cat data5.csv
result = pd.read_csv('data5.csv')
result
pd.isnull(result)

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [16]:
result = pd.read_csv('data5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [17]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('data5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### 1.2 分片/块读取文本数据

In [18]:
result = pd.read_csv('data6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
5,1.817480,0.742273,0.419395,-2.251035,Q
6,-0.776764,0.935518,-0.332872,-1.875641,U
7,-0.913135,1.530624,-0.572657,0.477252,K
8,0.358480,-0.497572,-0.367016,0.507702,S
9,-1.740877,-1.160417,-1.637830,2.172201,G


In [19]:
pd.read_csv('data6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [20]:
chunker = pd.read_csv('data6.csv', chunksize=100)
chunker

In [24]:
chunker = pd.read_csv('data6.csv', chunksize=100)

tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.order(ascending=False)

/Library/Python/2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: order is deprecated, use sort_values(...)


In [27]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

### 1.3 把数据写入文本格式

In [28]:
data = pd.read_csv('data5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [30]:
data.to_csv('out.csv')
!cat out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [31]:
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [32]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [33]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [34]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


In [36]:
dates = pd.date_range('1/1/2000', periods=7)
ts = Series(np.arange(7), index=dates)
ts.to_csv('tseries.csv')
!cat tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


In [37]:
Series.from_csv('tseries.csv', parse_dates=True)

2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
dtype: int64

### 1.4 手动读写数据（按要求）

In [38]:
!cat data7.csv

"a","b","c"
"1","2","3"
"1","2","3","4"


In [39]:
import csv
f = open('data7.csv')

reader = csv.reader(f)

In [40]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3', '4']


In [41]:
lines = list(csv.reader(open('data7.csv')))
header, values = lines[0], lines[1:]
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

In [42]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [43]:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

In [44]:
%cat mydata.csv

one;two;three
1;2;3
4;5;6
7;8;9


### 1.5 JSON格式的数据

In [64]:
obj = \
"""
{"姓名": "张三",
 "住处": ["天朝", "挖煤国", "万恶的资本主义日不落帝国"],
 "宠物": null,
 "兄弟": [{"姓名": "李四", "年龄": 25, "宠物": "汪星人"},
              {"姓名": "王五", "年龄": 23, "宠物": "喵星人"}]
}
"""

In [65]:
import json
result = json.loads(obj)
result

{u'\u4f4f\u5904': [u'\u5929\u671d',
  u'\u6316\u7164\u56fd',
  u'\u4e07\u6076\u7684\u8d44\u672c\u4e3b\u4e49\u65e5\u4e0d\u843d\u5e1d\u56fd'],
 u'\u5144\u5f1f': [{u'\u59d3\u540d': u'\u674e\u56db',
   u'\u5ba0\u7269': u'\u6c6a\u661f\u4eba',
   u'\u5e74\u9f84': 25},
  {u'\u59d3\u540d': u'\u738b\u4e94',
   u'\u5ba0\u7269': u'\u55b5\u661f\u4eba',
   u'\u5e74\u9f84': 23}],
 u'\u59d3\u540d': u'\u5f20\u4e09',
 u'\u5ba0\u7269': None}

In [66]:
print json.dumps(result, encoding="UTF-8", ensure_ascii=False)

{"兄弟": [{"年龄": 25, "宠物": "汪星人", "姓名": "李四"}, {"年龄": 23, "宠物": "喵星人", "姓名": "王五"}], "住处": ["天朝", "挖煤国", "万恶的资本主义日不落帝国"], "宠物": null, "姓名": "张三"}


In [67]:
result[u"兄弟"][0]

{u'\u59d3\u540d': u'\u674e\u56db',
 u'\u5ba0\u7269': u'\u6c6a\u661f\u4eba',
 u'\u5e74\u9f84': 25}

In [68]:
print json.dumps(result[u"兄弟"][0], encoding="UTF-8", ensure_ascii=False)

{"年龄": 25, "宠物": "汪星人", "姓名": "李四"}


In [69]:
asjson = json.dumps(result)

In [70]:
brothers = DataFrame(result[u'兄弟'], columns=[u'姓名', u'年龄'])
brothers

,姓名,年龄
0,李四,25
1,王五,23


### 1.6 人人都爱爬虫，人人都要解析XML 和 HTML

In [97]:
from lxml.html import parse
from urllib2 import urlopen

parsed = parse(urlopen('https://ask.julyedu.com/'))

doc = parsed.getroot()

In [98]:
doc

<Element html at 0x10d57bdb8>

In [99]:
links = doc.findall('.//a')
links[15:20]

[<Element a at 0x1121aa788>,
 <Element a at 0x1121aa7e0>,
 <Element a at 0x1121aa838>,
 <Element a at 0x1121aa890>,
 <Element a at 0x1121aa8e8>]

In [107]:
lnk = links[14]
lnk
lnk.get('href')
print lnk.text_content()

全部问题


In [108]:
urls = [lnk.get('href') for lnk in doc.findall('.//a')]
urls[-10:]

['https://ask.julyedu.com/people/July',
 'https://ask.julyedu.com/people/July',
 'http://weibo.com/askjulyedu',
 None,
 'https://www.julyedu.com/help/index/about',
 'https://www.julyedu.com/help/index/contact',
 'https://www.julyedu.com/help/index/join',
 'http://ask.julyedu.com/question/55',
 'http://www.julyapp.com',
 'http://www.miitbeian.gov.cn/']

In [130]:
spans = doc.findall('.//span')
len(spans)

137

In [132]:
def _unpack(spans):
    return [val.text_content() for val in spans]

In [134]:
contents = _unpack(spans)
for content in contents:
    print content

 通知设置
贡献
回复了问题 • 2 人关注 • 1 个回复 • 22 次浏览 • 2 小时前				
 •  来自相关主题
发起了问题 • 2 人关注 • 0 个回复 • 21 次浏览 • 22 小时前 
				
 •  来自相关主题
发起了问题 • 1 人关注 • 0 个回复 • 17 次浏览 • 1 天前 
				
 •  来自相关主题
贡献
回复了问题 • 3 人关注 • 1 个回复 • 38 次浏览 • 1 天前				
 •  来自相关主题
发起了问题 • 1 人关注 • 0 个回复 • 35 次浏览 • 1 天前 
				
 •  来自相关主题
发起了问题 • 1 人关注 • 0 个回复 • 29 次浏览 • 2 天前 
				
 •  来自相关主题
发起了问题 • 1 人关注 • 0 个回复 • 31 次浏览 • 2 天前 
				
 •  来自相关主题
贡献
回复了问题 • 37 人关注 • 36 个回复 • 484 次浏览 • 1 天前				
 •  来自相关主题
贡献
回复了问题 • 12 人关注 • 17 个回复 • 795 次浏览 • 2 天前				
 •  来自相关主题
贡献
回复了问题 • 4 人关注 • 4 个回复 • 197 次浏览 • 3 天前				
 •  来自相关主题
贡献
回复了问题 • 2 人关注 • 1 个回复 • 44 次浏览 • 3 天前				
 •  来自相关主题
贡献
回复了问题 • 2 人关注 • 1 个回复 • 34 次浏览 • 3 天前				
 •  来自相关主题
贡献
回复了问题 • 2 人关注 • 1 个回复 • 37 次浏览 • 3 天前				
 •  来自相关主题
贡献
回复了问题 • 2 人关注 • 1 个回复 • 102 次浏览 • 3 天前				
 •  来自相关主题
贡献
回复了问题 • 2 人关注 • 1 个回复 • 39 次浏览 • 4 天前				
 •  来自相关主题
发起了问题 • 1 人关注 • 0 个回复 • 26 次浏览 • 4 天前 
				
 •  来自相关主题
贡献
回复了问题 • 3 人关注 • 2 个回复 • 126 次浏览 • 4 天前				
 •  来自相关主题
发表了文章 • 1 个评论 • 764 次浏览 • 4 天前
 •  来

In [135]:
questions = doc.findall('.//h4')
len(questions)

50

In [136]:
contents = _unpack(questions)
for content in contents:
    print content


						@寒老师，CTR课程中的LR-MLlib的代码哪里下载?
					

						逻辑回归如何做特征选择？
					

						opencv在vs2015配置失败，有朋友能给一些可用的教程吗
					

						数据集具有倾斜性或呈长尾分布问题
					

						端到端深度学习在自动驾驶汽车上的应用
					

						CV班：转发微博狂送100元上课券，所有人 人人都有份
					

						关于word2vec的一些资料
					

						python数据分析班作业1
					

						第8类新班：《计算机视觉班》大纲讨论稿
					

						有奖竞答第二题（IBM ponder this 专题）：国王、硬币与电子秤的故事
					

						一个数学问题
					

						关于推荐系统基于内容的推荐中使用TF-IDF建模词对资料重要度问题
					

						python数据分析班-安装igraph后运行报错
					

						anaconda程序组中的各个组件都是做什么用的
					

						wide&deep learning问题
					

						SMO算法中关于b值的求解
					

						caffe车辆检测
					

						2017年各大互联网公司校招日程统计
					

						应用班第8次课 图像检索作业
					

						大事记：官网上线自动支付、七月在线APP全面开放下载
					

						趋势科技南京研发中心招聘机器学习、数据挖掘人才
					

						CNN 在文本中的应用
					

						关于特征工程的一些问题
					

						【DNN】为什么神经网络需要激励函数
					

						关于python的一个问题
					

						conda和pip install各种package有什么区别
					

						《python数据分析班》环境安装
					

						16年8月内推公司名单：VIPKID（算法负责人）、溢思得瑞（数据科学家）、洋钱罐（数据挖掘）
					

						一起来吐槽——对官网跟问答社区，有任何反馈或

### 1.7 解析XML

In [72]:
!head -21 Performance_MNR.xml

In [73]:
from lxml import objectify

path = 'Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

In [74]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [75]:
perf = DataFrame(data)
perf

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95
5,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,94.4,95,6,2008,96.2,95
6,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96,95,7,2008,96.2,95
7,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.4,95,8,2008,96.2,95
8,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,93.7,95,9,2008,95.9,95
9,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.4,95,10,2008,96,95


In [76]:
root

<Element PERFORMANCE at 0x108e692d8>

In [77]:
root.get('href')

In [78]:
root.text

## 二进制格式的数据

In [79]:
frame = pd.read_csv('data1.csv')
frame
frame.to_pickle('frame_pickle')

In [80]:
pd.read_pickle('frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### 使用HDF5格式

In [81]:
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

ImportError: HDFStore requires PyTables, "No module named tables" problem importing

In [ ]:
store['obj1']

In [ ]:
store.close()
os.remove('mydata.h5')

### HTML与API交互

In [82]:
import requests
url = 'https://api.github.com/repos/pydata/pandas/milestones/28/labels'
resp = requests.get(url)
resp

<Response [200]>

In [83]:
data[:5]

[{'AGENCY_NAME': 'Metro-North Railroad',
  'CATEGORY': 'Service Indicators',
  'DESCRIPTION': 'Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate service on these lines.\n',
  'FREQUENCY': 'M',
  'INDICATOR_NAME': 'On-Time Performance (West of Hudson)',
  'INDICATOR_UNIT': '%',
  'MONTHLY_ACTUAL': 96.9,
  'MONTHLY_TARGET': 95.0,
  'PERIOD_MONTH': 1,
  'PERIOD_YEAR': 2008,
  'YTD_ACTUAL': 96.9,
  'YTD_TARGET': 95.0},
 {'AGENCY_NAME': 'Metro-North Railroad',
  'CATEGORY': 'Service Indicators',
  'DESCRIPTION': 'Percent of commuter trains that arrive at their destinations within 5 minutes and 59 seconds of the scheduled time. West of Hudson services include the Pascack Valley and Port Jervis lines. Metro-North Railroad contracts with New Jersey Transit to operate service on these lines.\

In [84]:
issue_labels = DataFrame(data)
issue_labels

,AGENCY_NAME,CATEGORY,DESCRIPTION,FREQUENCY,INDICATOR_NAME,INDICATOR_UNIT,MONTHLY_ACTUAL,MONTHLY_TARGET,PERIOD_MONTH,PERIOD_YEAR,YTD_ACTUAL,YTD_TARGET
0,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,1,2008,96.9,95
1,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95,95,2,2008,96,95
2,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.9,95,3,2008,96.3,95
3,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,98.3,95,4,2008,96.8,95
4,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,95.8,95,5,2008,96.6,95
5,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,94.4,95,6,2008,96.2,95
6,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96,95,7,2008,96.2,95
7,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.4,95,8,2008,96.2,95
8,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,93.7,95,9,2008,95.9,95
9,Metro-North Railroad,Service Indicators,Percent of commuter trains that arrive at thei...,M,On-Time Performance (West of Hudson),%,96.4,95,10,2008,96,95


## 2.数据库相关操作

## 2.1 sqlite数据库

In [85]:
import sqlite3

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""

con = sqlite3.connect(':memory:')
con.execute(query)
con.commit()

In [86]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)
con.commit()

In [87]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[(u'Atlanta', u'Georgia', 1.25, 6),
 (u'Tallahassee', u'Florida', 2.6, 3),
 (u'Sacramento', u'California', 1.7, 5)]

In [88]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [89]:
DataFrame(rows, columns=zip(*cursor.description)[0])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [90]:
import pandas.io.sql as sql
sql.read_sql('select * from test', con)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


### 3.2 MySQL数据库

In [ ]:
#coding=utf-8
import MySQLdb

conn= MySQLdb.connect(
        host='localhost',
        port = 3306,
        user='root',
        passwd='123456',
        db ='test',
        )
cur = conn.cursor()

#创建数据表
#cur.execute("create table student(id int ,name varchar(20),class varchar(30),age varchar(10))")

#插入一条数据
#cur.execute("insert into student values('2','Tom','3 year 2 class','9')")


#修改查询条件的数据
#cur.execute("update student set class='3 year 1 class' where name = 'Tom'")

#删除查询条件的数据
#cur.execute("delete from student where age='9'")

cur.close()
conn.commit()
conn.close()

### 3.3 Memcache

In [ ]:
#coding:utf8
import memcache

class MemcachedClient():
    ''' python memcached 客户端操作示例 '''

    def __init__(self, hostList):
        self.__mc = memcache.Client(hostList);

    def set(self, key, value):
        result = self.__mc.set("name", "NieYong")
        return result

    def get(self, key):
        name = self.__mc.get("name")
        return name

    def delete(self, key):
        result = self.__mc.delete("name")
        return result

if __name__ == '__main__':
    mc = MemcachedClient(["127.0.0.1:11511", "127.0.0.1:11512"])
    key = "name"
    result = mc.set(key, "NieYong")
    print "set的结果：", result
    name = mc.get(key)
    print "get的结果：", name
    result = mc.delete(key)
    print "delete的结果：", result

### 3.4 MongoDB

In [ ]:
#encoding:utf=8  
import pymongo  
  
connection=pymongo.Connection('10.32.38.50',27017)  
  
#选择myblog库  
db=connection.myblog  
  
# 使用users集合  
collection=db.users  
  
# 添加单条数据到集合中  
user = {"name":"cui","age":"10"}  
collection.insert(user)  
  
#同时添加多条数据到集合中  
users=[{"name":"cui","age":"9"},{"name":"cui","age":"11"}]  
collection.insert(users)  
  
#查询单条记录  
print collection.find_one()  
  
#查询所有记录  
for data in collection.find():  
    print data  
  
#查询此集合中数据条数  
print collection.count()  
  
#简单参数查询  
for data in collection.find({"name":"1"}):  
    print data  
  
#使用find_one获取一条记录  
print collection.find_one({"name":"1"})  
  
  
#高级查询  
print "__________________________________________"  
print '''''collection.find({"age":{"$gt":"10"}})'''  
print "__________________________________________"  
for data in collection.find({"age":{"$gt":"10"}}).sort("age"):  
    print data  
  
# 查看db下的所有集合  
print db.collection_names()  